<a href="https://colab.research.google.com/github/wFairmanOlin/HAUCS/blob/main/GPS_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
import pandas as pd
from datetime import datetime
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import plotly.express as px

In [2]:
### ONLY RUN ONCE! ###
# #download data from firebase
# cred = credentials.Certificate("/content/haucs-monitoring-firebase-adminsdk-wt1pb-1291f2083e.json")
# firebase_admin.initialize_app(cred, {'databaseURL': 'https://haucs-monitoring-default-rtdb.firebaseio.com'})
try:
    print(cred)
except:
    cred = credentials.Certificate("../Biomass/pc_basestation/fb_key.json")
    firebase_admin.initialize_app(cred, {'databaseURL': 'https://haucs-monitoring-default-rtdb.firebaseio.com'})

In [3]:
## ONLY RUN TO REFRESH DATA! ###
ref = db.reference('/')
data = ref.get()

### RUN TO SAVE JSON FILE LOCALLY ###
with open('latest_data.json', 'w') as f:
    json.dump(data, f)

In [4]:
gps_df = pd.DataFrame(data['gps'], index=['RSSI', 'lat', 'lng']).T.reset_index()
gps_df.columns = ['timestamp', 'RSSI', 'lat', 'lng']

In [5]:
# Split the concatenated values in the lat and lng columns
gps_df[['lat', 'lng']] = gps_df[['lat', 'lng']].apply(lambda x: x.str.split().str[0])

# Convert the necessary columns to numeric type
gps_df['RSSI'] = pd.to_numeric(gps_df['RSSI'])
gps_df['lat'] = pd.to_numeric(gps_df['lat'])
gps_df['lng'] = pd.to_numeric(gps_df['lng'])

In [37]:
# Convert the timestamp column to datetime type
gps_df['timestamp'] = pd.to_datetime(gps_df['timestamp'], format='%Y%m%d_%H:%M:%S')

# Create the geographical heatmap plot using Plotly Express
fig = px.density_mapbox(gps_df, lat='lat', lon='lng', z='RSSI', radius=10, mapbox_style='carto-positron')

# Set mapbox token and layout
fig.update_layout(
    mapbox=dict(
        accesstoken='YOUR_MAPBOX_TOKEN',
        center=dict(lat=gps_df['lat'].mean(), lon=gps_df['lng'].mean()),
        zoom=10
    )
)

# Display the plot
fig.show()

In [48]:
# px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(gps_df,
                        lat="lat",
                        lon="lng",
                        hover_name="RSSI",
                        color="RSSI",
                        mapbox_style='carto-positron',
                        zoom=1)

# Set mapbox token and layout
fig.update_layout(
    width=1000,
    height=600,
    mapbox=dict(
        accesstoken='YOUR_MAPBOX_TOKEN',
        center=dict(lat=gps_df['lat'].mean(), lon=gps_df['lng'].mean()),
        zoom=12
    )
)

fig.show()
